<a href="https://colab.research.google.com/github/SilPrad0/Writing-Assistant/blob/main/NovelEditor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytorch transformers datasets pandas

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pytorch)


In [5]:
!git clone https://github.com/huggingface/datasets.git
%cd datasets
!pip install .

Cloning into 'datasets'...
remote: Enumerating objects: 67089, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 67089 (delta 76), reused 37 (delta 34), pack-reused 66986 (from 4)
Receiving objects: 100% (67089/67089), 87.13 MiB | 12.44 MiB/s, done.
Resolving deltas: 100% (32902/32902), done.
/content/datasets
Processing /content/datasets
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for datasets: filename=datasets-3.2.1.dev0-py3-none-any.whl size=481765 sha256=416de67900da563c0d54616af880ddd518efc5381a024685c7f756165e3c8f45
  Stored in directory: /tmp/pip-ephem-wheel-cache-awya4_8m/wheels/0a/ce/6e/0de7571b8c16c1829fd4a2b729902ca4437dbf307610fe8fed
Successfully built datasets
  Attempting uninstall: datasets
    Found existing installation: datasets 3.2.0
    Uninstalling datasets-3.2.0:
      Successfully uninstalle

In [12]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.9 MB/s eta 0:00:00


In [15]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=3f06d2430ebcd26936f7e4711e9a919501005674e40b46c6e48a9dc5fbd6c9f5
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [17]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import json
import wandb
import pandas as pd
import evaluate

# Initialize W&B project
wandb.login()
wandb.init(project="your_project_name", entity="your_wandb_username")

# Load JSON
with open('/content/sample_data/data.json', 'r') as f:
    data = json.load(f)

# Convert to a list of dicts for DataFrame
scene_data = []
for scene in data["scenes"]:
    scene_data.append({"input_text": scene["input_text"], "output_text": scene["output_text"]})

# Convert to Hugging Face Dataset
df = pd.DataFrame(scene_data)
hf_dataset = Dataset.from_pandas(df)
hf_dataset = hf_dataset.train_test_split(test_size=0.1)
train_dataset = hf_dataset["train"]
eval_dataset = hf_dataset["test"]

# Load Tokenizer and Model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Preprocessing
def preprocess_function(examples):
    inputs = ["Polish: " + text for text in examples["input_text"]]
    outputs = examples["output_text"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(outputs, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=10,
    logging_strategy="steps",
    report_to="wandb",  # Log metrics to W&B
)

# Load the metric
metric = evaluate.load("rouge")

# Your compute_metrics function remains the same
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return metric.compute(predictions=decoded_preds, references=decoded_labels)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Train
trainer.train()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


TrainOutput(global_step=6, training_loss=12.768656412760416, metrics={'train_runtime': 90.8405, 'train_samples_per_second': 0.132, 'train_steps_per_second': 0.066, 'total_flos': 1624101617664.0, 'train_loss': 12.768656412760416, 'epoch': 3.0})

In [18]:
model.save_pretrained("./my_model")
tokenizer.save_pretrained("./my_model")


('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/spiece.model',
 './my_model/added_tokens.json')